# Get Atlas Release Dataset

## Imports

In [62]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
import pprint
from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.resources import Dataset

## Setup
Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


In [9]:
TOKEN = ""

In [87]:
endpoint_staging = "https://staging.nise.bbp.epfl.ch/nexus/v1"
endpoint_prod = "https://bbp.epfl.ch/nexus/v1"
endpoint=endpoint_prod

forge = KnowledgeGraphForge("prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint=endpoint, 
                            bucket="bbp/atlas")

In [ ]:
# Install dependencies
!pip install jsonpath_ng

## Set the atlas release id
These atlas releases can be explored through the atlas web app:

* dev: https://bluebrainatlas.kcpdev.bbp.epfl.ch/atlas
* prod: https://bbp.epfl.ch/atlas


In [88]:

Prod_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/4906ab85-694f-469d-962f-c0174e901885" 
Prod_METype_DENSITY_Release_Id = "https://bbp.epfl.ch/neurosciencegraph/data/27652d4d-3a6f-42c7-9833-64396104c445"
Prod_METype_DENSITY_Release_Version="v0.3.2"
Staging_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/brainatlasrelease/c96c71a8-4c0d-4bc1-8a1a-141d9ed6693d" 

atlas_release_id = Prod_BBP_Mouse_Brain_Atlas_Release

## Get the atlas release high level metadata

In [89]:
atlas_release = forge.retrieve(atlas_release_id)

In [ ]:
print(atlas_release)

In [90]:
# Get the current revision of the Atlas release
atlas_release._store_metadata["_rev"]

2

## Get the atlas hierarchy

In [91]:
parcellation_ontology = forge.retrieve(atlas_release.parcellationOntology.id, cross_bucket=True)

In [ ]:
print(parcellation_ontology)

In [92]:
parcellation_ontology_copy = Dataset.from_resource(forge, parcellation_ontology, store_metadata=True)
parcellation_ontology_copy.distribution = [d for d in parcellation_ontology.distribution if d.encodingFormat == "application/json"]

forge.download(parcellation_ontology_copy, "distribution.contentUrl", ".", overwrite=True, cross_bucket=True)

## Get a brain region metadata from it's name or accronym

In [93]:
name_accronym = "Medial septal nucleus"

In [94]:
from kgforge.core.commons.strategies import ResolvingStrategy
brain_region = forge.resolve(name_accronym, scope="ontology", target="terms", strategy=ResolvingStrategy.EXACT_CASEINSENSITIVE_MATCH)
print(brain_region)

{
    id: http://api.brain-map.org/api/v2/data/Structure/564
    type: Class
    label: Medial septal nucleus
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/core/brainregion
    notation: MS
    prefLabel: Medial septal nucleus
    subClassOf: nsg:BrainRegion
}


In [95]:
# Get accronym
brain_region.notation

'MS'

## Get parcellation (annotation) volume

In [96]:
parcellation_volume = forge.retrieve(atlas_release.parcellationVolume.id)

In [ ]:
print(parcellation_volume)

In [97]:
forge.download(parcellation_volume, "distribution.contentUrl", ".", overwrite=True)

## Get orientation field and Direction vectors volumes

In [98]:
query = {
          "type":"CellOrientationField", 
          "atlasRelease":{"@id":atlas_release_id},
          "brainLocation":{"brainRegion":{"id":"http://api.brain-map.org/api/v2/data/Structure/997"}} # root brain region
        }
cell_orientation_field = forge.search(query)
print(f"{len(cell_orientation_field)} found")

2 found


In [ ]:
print(cell_orientation_field)

In [99]:
forge.download(cell_orientation_field, "distribution.contentUrl", ".", overwrite=True)

## Get the released metype densities

In [100]:
metype_density_release = forge.retrieve(id=Prod_METype_DENSITY_Release_Id,
                                     version = Prod_METype_DENSITY_Release_Version,
                                     cross_bucket=True)

In [101]:
forge.download(metype_density_release,path=".", follow="distribution.contentUrl", cross_bucket=True, overwrite=True) # downloads a json file containing released metype densities along with their version

In [102]:
with open(f"./{metype_density_release.distribution.name}", "r") as f:
    metype_density_release_json= json.load(f)

In [103]:
from jsonpath_ng import jsonpath, parse

jsonpath_expr = parse('hasPart[*].hasPart[*].hasPart[*].@id')
metype_density_release_volume_ids = [(match.value, match.context.value["_rev"]) for match in jsonpath_expr.find(metype_density_release_json)] # yields [(metype volume id, revision)]
metype_density_release_volumes = [forge.retrieve(id=volume_id[0], version=volume_id[1], cross_bucket=True) for volume_id in metype_density_release_volume_ids] # metype density volumes

len(metype_density_release_volumes)

245

In [104]:
# Collect metadata as pandas dataframe 
reshaped_resources = forge.reshape(metype_density_release_volumes, keep=["id","type", "annotation.hasBody.id", "annotation.hasBody.label",
                     "brainLocation.brainRegion.id", "brainLocation.brainRegion.label", "distribution.atLocation.location"])
df = forge.as_dataframe(reshaped_resources, nesting=".")

df["mtype"] = df.apply(lambda row: (row.annotation[0]["hasBody"]["label"],row.annotation[0]["hasBody"]["id"]) , axis=1) 
df["etype"] = df.apply(lambda row: (row.annotation[1]["hasBody"]["label"],row.annotation[1]["hasBody"]["id"]) , axis=1) 
type_column = df.pop('type')
mtype_column = df.pop('mtype')
etype_column = df.pop('etype')

df.insert(0, 'type', type_column)
df.insert(1, 'mtype', mtype_column)
df.insert(2, 'etype', etype_column)
df.drop(columns="annotation")

df.head(100)

,type,mtype,etype,id,annotation,brainLocation.brainRegion.id,brainLocation.brainRegion.label,distribution.atLocation.location
0,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L23_BP, http://uri.interlex.org/base/ilx_0383...","(dSTUT, http://uri.interlex.org/base/ilx_0738202)",https://bbp.epfl.ch/neurosciencegraph/data/88e...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
1,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L23_BP, http://uri.interlex.org/base/ilx_0383...","(cNAC, http://uri.interlex.org/base/ilx_0738201)",https://bbp.epfl.ch/neurosciencegraph/data/9ab...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
2,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L23_BP, http://uri.interlex.org/base/ilx_0383...","(bAC, http://uri.interlex.org/base/ilx_0738199)",https://bbp.epfl.ch/neurosciencegraph/data/2df...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
3,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L23_BP, http://uri.interlex.org/base/ilx_0383...","(bIR, http://uri.interlex.org/base/ilx_0738206)",https://bbp.epfl.ch/neurosciencegraph/data/6e2...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
4,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L23_BP, http://uri.interlex.org/base/ilx_0383...","(bNAC, http://uri.interlex.org/base/ilx_0738203)",https://bbp.epfl.ch/neurosciencegraph/data/a40...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
...,...,...,...,...,...,...,...,...
95,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L4_LBC, http://uri.interlex.org/base/ilx_0383...","(bAC, http://uri.interlex.org/base/ilx_0738199)",https://bbp.epfl.ch/neurosciencegraph/data/104...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
96,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L4_LBC, http://uri.interlex.org/base/ilx_0383...","(cNAC, http://uri.interlex.org/base/ilx_0738201)",https://bbp.epfl.ch/neurosciencegraph/data/699...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
97,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L4_LBC, http://uri.interlex.org/base/ilx_0383...","(cAC, http://uri.interlex.org/base/ilx_0738197)",https://bbp.epfl.ch/neurosciencegraph/data/045...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...
98,"[NeuronDensity, VolumetricDataLayer, CellDensi...","(L4_LBC, http://uri.interlex.org/base/ilx_0383...","(bSTUT, http://uri.interlex.org/base/ilx_0738200)",https://bbp.epfl.ch/neurosciencegraph/data/0cb...,[{'hasBody': {'id': 'http://uri.interlex.org/b...,http://api.brain-map.org/api/v2/data/Structure...,root,file:///gpfs/bbp.cscs.ch/data/project/proj39/n...


In [105]:
forge.download(metype_density_release_volumes, path="./test", follow="distribution.contentUrl", cross_bucket=True, overwrite=True)